<a href="https://colab.research.google.com/github/Hiennoob123/Simple-game-for-life/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This problem is based on the official gym environment provided in https://gymnasium.farama.org/environments/classic_control/pendulum/

Since the official version has continuous state space and continuous action space, which requires new techniques to handle that is out side the scope of this course, we discretize the state space and action space. The discretized version is provided in the following package.

In [ ]:
!pip install -q kaist-or-gym

The following code creates a pendulum environment.

In [ ]:
import gymnasium as gym
import kaist_or_gym
from kaist_or_gym.envs.pendulum import DiscretePendulumEnv

env = DiscretePendulumEnv(render_mode="human")

Run the following code to visualize an episode generated by a random policy.

In [ ]:
import time

obs, info = env.reset()
for step in range(200):
    # Sample a random action (uniform over 4 directions)
    action = env.action_space.sample()
    # Perform the action
    obs, reward, terminated, truncated, info = env.step(action)

    env.render()

    if terminated or truncated:
        print(f"Reached goal at step {step}.")
        break

The reward is designed so that the inverted (upright) position of the pendulum is the desired state. See https://gymnasium.farama.org/environments/classic_control/pendulum/ for the full description of the reward function.

Here is a function introduced in lecture that runs a single value iteration step using probability matrix P and current value function Q and discounting factor gamma.

In [ ]:
import numpy as np

def single_value_iteration(P, Q, gamma):
    """
    Arguments:
    - P: A dictionary mapping (s, a) → {next_state: probability}.
        If (s, a) is missing, the next state is assumed to be drawn uniformly
        from all states.
    - Q: Current Q-value matrix of shape (n_states, n_actions).
    - gamma: Discount factor.
    """

    Q_new = np.zeros(Q.shape, dtype=float)
    V = Q.max(axis=1)
    for s in range(n_states):
        for a in range(n_actions):
            if (s, a) not in P:
                Q_new[s, a] = env.reward(s, a) + gamma * np.mean(V)
            else:
                Q_new[s, a] = env.reward(s, a)
                for next_state, prob in P[s, a].items():
                    Q_new[s, a] += prob * gamma * V[next_state]
    return Q_new


Here is a function introduced in lecture that computes empirical (estimated) transition probability given a dataset of transitions.

In [ ]:
def compute_empirical_transition_probabilities(D):
    counts = {}  # (s, a) -> {s': count}
    for s, a, r, s_prime in D:
        if (s, a) not in counts:
            counts[(s, a)] = {}
        counts[(s, a)][s_prime] = counts[(s, a)].get(s_prime, 0) + 1

    empirical_probs = {}  # (s, a) -> {s': prob}
    for (s, a), s_prime_counts in counts.items():
        total_transitions = sum(s_prime_counts.values())
        if total_transitions > 0:
            empirical_probs[(s, a)] = {s_prime: count / total_transitions for s_prime, count in s_prime_counts.items()}
    return empirical_probs

The following function runs an episode using a policy (optionally with noise for ε-greedy exploration) and returns a list of transitions.

In [ ]:
def run_one_episode(env, H: int, policy=None, noise=0):
    D = []
    s, _ = env.reset()
    for _ in range(H):
        if policy is None or np.random.rand() < noise:
            a = env.action_space.sample()
        else:
            a = policy[s]
        s_prime, reward, terminated, truncated, _ = env.step(a)
        D.append((s, a, reward, s_prime))
        s = s_prime
        if terminated or truncated:
            break
    return D

Here is a skeleton code introduced in lecture that runs value iteration. Please complete the code.

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

# you may have to tweak these
K = 50 # number of episodes
H = 50 # episode length

# compute epsilon to be used in epsilon greedy method
# you may have to tweak this function.
def get_epsilon(k):
    return 0.5 ** k

gamma = 0.99
n_states = env.observation_space.n
n_actions = env.action_space.n
Q = np.zeros((n_states, n_actions), dtype=float)

D = []
reward = []

pbar = tqdm(range(K))
for k in pbar:
    # In lecture, we ran a "full" value iteration here until the Q function stabilizes.
    # However, running the "full" value iteration every episode is computationally inefficient.
    # Instead, run a "single" value iteration step to update the Q function once.

    # TODO: compute transition probability using the dataset D
    # TODO: run a single value iteration to update the Q value function.
    policy = Q.argmax(axis=1)

    epsilon = get_epsilon(k)
    trajectory = run_one_episode(env, H, policy, epsilon)
    D.extend(trajectory)
    ret = sum(r for _, _, r, _ in trajectory)
    reward.append(ret)
    pbar.set_postfix({
        "reward": ret,
        "epsilon": epsilon,
    })

# Assuming episode_length list is available from previous execution
episode_numbers = range(len(reward))

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.plot(episode_numbers, reward, color='blue', label='Reward')
ax1.set_xlabel('Episode Number')
ax1.set_ylabel('Reward', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis for epsilon
ax2 = ax1.twinx()
ax2.plot(episode_numbers, [get_epsilon(k) for k in range(K)], color='red', linestyle='--', label='Epsilon')
ax2.set_ylabel('Epsilon Value', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Reward and Epsilon vs. Episode Number')
fig.tight_layout() # Adjust layout to prevent overlapping labels
plt.grid(True)
plt.show()

Here is code for testing the policy learned. It saves the animation into a gif file and downloads it onto your computer.

In [ ]:
import time
import imageio
import matplotlib.pyplot as plt

frames = []

obs, info = env.reset(seed=4)
for step in range(200):
    action = Q.argmax(axis=1)[obs]
    obs, reward, terminated, truncated, info = env.step(action)

    env.render()

    fig = env.fig
    fig.canvas.draw()
    buf = np.asarray(fig.canvas.buffer_rgba()).copy()
    frame = buf[:, :, :3]
    frames.append(frame)

gif_path = "trajectory.gif"
imageio.mimsave(gif_path, frames, fps=16)

from google.colab import files
files.download("trajectory.gif")

Your task is to modify the value iteration algorithm so that it learns a good policy. You may need to adjust the number of episodes, the episode length, and the schedule for decaying ε. The resulting policy should be able to stabilize the inverted pendulum. Please submit the GIF file for your homework; the code does not need to be submitted.